In [31]:
import matplotlib.pyplot as plt
import math
import random
from math import sqrt
# importing Qiskit
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.circuit.library import GroverOperator
from qiskit.algorithms import AmplificationProblem,Grover
# importing backend 
from typing import List
from qiskit.utils import QuantumInstance
# importing Grover 
from GroverFlowShop import GroverFlowSop
import numpy as np

In [32]:
class GroverOptimizerFSP :
    def __init__(
        self,
        num_jobs : int,
        num_machines : int,
        process_time : List[List[int]],
        upper : int,
        quantum_instance : QuantumInstance,
        num_iteration  
        )-> None :
        """
            Args : 
            problem : flow shop instance {m,n,processsiog time matrix}
            quantum instance : backend to execute the circuit
            num_iteration : number of iteration if there is no improvement
        """
        self.num_jobs = num_jobs
        self.num_machines = num_machines
        self.process_time = process_time
        self.upper = upper
        self.quantum_instance = quantum_instance
        self.num_iteration = num_iteration


    def solve(self):
        """
            GAS solver 
        """
        # Optimum tracking variables
        optimum_found = False
        optimum_permu = math.inf
        optimum_value = math.inf

        # Grover ciruit parameters
        upperBound = self.upper
        n = self.num_jobs
        N = 2**n
        m = self.num_machines
        q = self.qubits_cmj_Estimation()
        pm = self.process_time       
        quantum_instance = self.quantum_instance
        # solotions tracking
        num_solutions = 2**(n*(2**n))
        iteration = 0

        # Grover result
        op_counts = {}
        iteration = 0

        # Stop the algorithm if we hit the rotation max
        r = 0 
        max_r = int(math.ceil(100 * math.pi / 4))

        while not optimum_found :
            rm = 1
            impovement = False

            # Iterate until we don't get improvement
            nb_no_improvement = 0
            while not impovement :
                # The required number of rotation 
                nb_no_improvement += 1
                r_count = random.randint(0,m)
                r += r_count

                # Create Grover FSP Circuit
                grover = GroverFlowSop(n,q,pm,m,upperBound,r_count,quantum_instance)

                # Execute Grover 
                grover_output = grover.execute()

                # Choose a random solution from grover results
                output = random.choice(list(grover_output.items()))
                k = grover.convert_solution_int(output)
                cmax = self.calculate_makespan(np.array(k))
                if cmax < optimum_value :
                    optimum_value = cmax
                    optimum_permu = k
                    impovement = True
                    upperBound = cmax
                else:
                    m = int(np.ceil(min(m*8/7,2**(n*N/2))))    
                    if (nb_no_improvement >= self.num_iteration or r >= max_r):
                        impovement_found = True
                        optimum_found = True
            
    def qubits_cmj_Estimation(self) ->int :
        """
            Estimate the required number of qubits for the circuit
         """
        n = self.num_jobs
        m = self.num_machines
        return math.ceil(math.log2(sum(self.process_time[i][j] for i in range(m) for j in range(n) )))

    def calculate_makespan(self,a, seq):
        a = np.transpose(a)
        # Order the jobs (rows) in order of the sequence
        a = a[seq]

        b = np.zeros(a.shape)
        jobs = a.shape[0]
        macs = a.shape[1]

        b[0, 0] = a[0, 0]
        # Build first row
        for i in range(1, macs):
            b[0, i] = a[0, i] + b[0, i - 1]
        # Build first column
        for i in range(1, jobs):
            b[i, 0] = a[i, 0] + b[i - 1, 0]
        # Build the rest
        for i in range(1, jobs):
            for j in range(1, macs):
                b[i, j] = a[i, j] + (b[i - 1, j] if b[i - 1, j] > b[i, j - 1] else b[i, j - 1])

        return int(b[-1, -1])
       
GAS = GroverOptimizerFSP(2,2,[[1,2],[3,4]],14,None,1)
q = GAS.qubits_cmj_Estimation()
q


4